# ResNet Activations - 1m

In [1]:
BASE_RES = 1

In [3]:
from keras.applications.resnet50 import ResNet50, preprocess_input
#from keras.applications.resnet import ResNet101, preprocess_input
from keras.models import Model
from keras.preprocessing import image
from keras import models
from keras import layers
from keras import optimizers
from keras.models import model_from_json

In [4]:
from matplotlib import pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd
import sys
import os
import datetime

from PIL import Image

from dotenv import load_dotenv

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [5]:
sys.path.append("../Library/")
import deep_learning as dl
import image_manipulation as ima
import machine_learning as ml

In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
def print_time(string=''):
    print('', str(datetime.datetime.today())[:22], string)
    return

# Load Image Batch

### Define degraded resolutions and image sizes for later use

In [8]:
sizes = {}
base_res = BASE_RES
base_size = 512

for factor in range(2, 17):
    res = round(base_res * factor,1)
    size = round(base_size/factor)
    sizes[res] = (size, size)
print("\nSizes dictionary:\n", sizes)


Sizes dictionary:
 {2: (256, 256), 3: (171, 171), 4: (128, 128), 5: (102, 102), 6: (85, 85), 7: (73, 73), 8: (64, 64), 9: (57, 57), 10: (51, 51), 11: (47, 47), 12: (43, 43), 13: (39, 39), 14: (37, 37), 15: (34, 34), 16: (32, 32)}


### Load Image Batch into DataFrame

In [9]:
base_folder = os.getenv('GDRIVE_FOLDER') + "MFP - Satellogic/images/usgs_512_res" + str(BASE_RES) + "m"
categories = ["forest-woodland", "agriculture", "shrubland-grassland", "semi-desert"]
data_folder_colab = os.getenv('GDRIVE_FOLDER').replace("MDS/","") + "Colab/MasterThesis/Data/" + str(BASE_RES) + "m"
results_folder_colab = os.getenv('GDRIVE_FOLDER').replace("MDS/","") + "Colab/MasterThesis/Results_" + str(BASE_RES) + "m_excl_label1"

In [10]:
labels = [0, 1, 2]
df_images = pd.DataFrame(columns = ['filename', 'image', 'resolution', 'label', 'category'])
for category in categories:
    for label in labels:
        df = ima.load_images_into_df_by_category_and_label(base_folder, category, label)
        df_images = df_images.append(df)    

In [11]:
print("Shape of df_images:", df_images.shape)
print("Distribution of categories and labels:\n", df_images.groupby(['category', 'label']).size().reset_index(name='counts'))
print("\nDistribution of labels:\n", df_images.groupby(['label']).size().reset_index(name='counts'))

Shape of df_images: (238, 5)
Distribution of categories and labels:
               category  label  counts
0          agriculture      1       2
1          agriculture      2       8
2      forest-woodland      0      66
3      forest-woodland      1      39
4      forest-woodland      2      79
5          semi-desert      0      14
6          semi-desert      1      10
7          semi-desert      2       6
8  shrubland-grassland      0      14

Distribution of labels:
    label  counts
0      0      94
1      1      51
2      2      93


### Enhance Images DataFrame by Degraded Images

In [10]:
if False:

    df_degraded = ima.load_degraded_images_into_df(df_images, sizes, label = True)

    df_images = df_images.append(df_degraded)
    print("Shape of df_images:", df_images.shape)
    del df_degraded

### Generate data set X and y and save for Colab

In [12]:
X, y = dl.generate_X_y_from_df(df_images)

Shape of image array is: (238, 512, 512, 3)


In [15]:
X_l1, y_l1 = X[y == 1], y[y == 1] 
X_l1

array([[[[ 59,  84,  61],
         [ 64,  87,  65],
         [ 68,  91,  66],
         ...,
         [ 56,  69,  54],
         [ 67,  89,  61],
         [ 68,  88,  63]],

        [[ 57,  81,  58],
         [ 70,  96,  67],
         [ 72, 100,  69],
         ...,
         [ 62,  79,  61],
         [ 69,  85,  61],
         [ 55,  77,  58]],

        [[ 66,  94,  64],
         [ 79, 104,  71],
         [ 87, 112,  76],
         ...,
         [ 36,  43,  46],
         [ 22,  34,  41],
         [ 28,  40,  45]],

        ...,

        [[ 31,  47,  46],
         [ 17,  30,  38],
         [ 20,  35,  39],
         ...,
         [ 51,  68,  54],
         [ 23,  39,  42],
         [ 13,  22,  34]],

        [[ 38,  54,  48],
         [ 28,  42,  43],
         [ 37,  56,  48],
         ...,
         [ 51,  74,  56],
         [ 46,  68,  51],
         [ 17,  23,  35]],

        [[ 26,  45,  44],
         [ 40,  56,  49],
         [ 53,  73,  55],
         ...,
         [ 65,  87,  60],
        

In [16]:
from random import shuffle
random_order = [i for i in range(len(X))]
shuffle(random_order)

X, y = X[random_order], y[random_order]

not_label = y != 1
X, y = X[not_label], y[not_label]
y = np.array(ml.convert_encoding(y, ml.zero_encoding))

In [17]:
X_name = 'X_images_res' + str(BASE_RES) + '.npy'
y_name = 'y_images_res' + str(BASE_RES) + '.npy'

In [18]:
np.save(os.path.join(data_folder_colab, X_name), X)
np.save(os.path.join(data_folder_colab, y_name), y)

In [20]:
X_l1_name = 'X_l1_images_res' + str(BASE_RES) + '.npy'
y_l1_name = 'y_l1_images_res' + str(BASE_RES) + '.npy'

np.save(os.path.join(results_folder_colab, X_l1_name), X_l1)
np.save(os.path.join(results_folder_colab, y_l1_name), y_l1)